# ZPEM2509 Astrophysics Lab: Eclipsing Binaries 


*Developed by Dr Simon Murphy, UNSW Canberra ([simon.murphy@unsw.edu.au](mailto:simon.murphy@unsw.edu.au))* 

<br>
<div style="background-color:#eee; padding:10px; border:1px solid black; border-radius:5px"><p>In this lab you will analyse real observations of an eclipsing binary star using an online notebook written in Python (<em>don’t worry, no programming or Python experience is required, just a web browser</em>). This notebook will step you through a number of activities and questions to answer. Please have the background reading handy as we will refer to it during the lab. There are also two pre-lab questions which should take &lt;15 min to answer.</p>

<p>You may work in pairs but should submit individual reports by the end of the lab session. Your report should include all plots and calculations, and should answer all the questions in the pre-lab and lab activities, with enough detail so that you or a colleague could recreate your analysis.</p>
</div>

<br>


# 1. The young eclipsing binary THOR 42


Our target in this lab is the eclipsing binary "THOR 42", which is the 42nd member of a young group of stars called the 32 Orionis (Th Or) Moving Group. THOR 42 comprises two young M-type stars which are smaller, less massive, fainter and cooler than the Sun. __Your goal is to use the light and radial velocity curves of THOR 42 to estimate the radii and masses of both stars__. 

<div style="text-align:center;margin:30px">
    <table><tr>
        <td><img src="orion.png" style="height:350px"></td>
        <td><img src="THOR42.jpg" style="height:350px"></td>
    </tr></table>
    Location of THOR 42 in the constellation of Orion, to the east of Orion's belt <br>(remember that east and west are swapped when looking out at the sky from the Earth).
</div>

THOR 42 was observed by the NASA *TESS* ([*Transiting Exoplanet Survey Satellite*](https://www.nasa.gov/tess-transiting-exoplanet-survey-satellite)) mission between 2018 December 15 and 2019 January 6 as part of its all-sky survey.  We will use *TESS* data to construct a light curve for THOR 42, then analyse radial velocity measurements from the [ANU 2.3-m telescope](https://rsaa.anu.edu.au/observatories/telescopes/anu-23m-telescope) at [Siding Spring Observatory](https://goo.gl/maps/UBrdy1XS57sKuqrR9) near Coonabarabran, NSW. 


-----
First, let's load some useful Python modules to read the data and make plots. 

Click in the code cell so it turns green and press `SHIFT` + `RETURN` to run the code. If everything works you will receive a message saying a session has started and the focus will move to the next cell.

<br>


In [ ]:
# Import various libraries
%matplotlib widget
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from interactive_figures import RVCurve, LightCurve, MassRadiusDiagram
import mybinder
mybinder.start_session()

-------

# 2. Light curve analysis 

We begin by loading the *TESS* photometry from a text file. This file has 3 columns - the  *Barycentric Julian Date* (BJD - 2450000), the flux (brightness) from the *TESS* pixel containing THOR 42 and a flux error which we can ignore. The first few rows of the file look like this:


In [ ]:
!head TESS.txt # Run the Unix 'head' command to show the first 10 lines of the file


The light curve has already been normalised so that maximum light corresponds to a flux of approximately $1.0$ (in arbitrary units). Rather than Earth-centred Julian Dates (JD; the decimal number of days since noon on January 1, 4713 BC), we use *Barycentric Julian Dates* (BJD) calculated at the centre-of-mass of the solar system (approximately the centre of the Sun). This correction is necessary because otherwise the times recorded by *TESS* would be up to $\pm$8 min different depending on which side of the Sun *TESS* was at the time of observation.

Lets load the file and pull out the BJD times and fluxes:


In [ ]:
# Load the TESS photometry
tess = Table.read('TESS.txt', format='ascii')
bjd = tess['BJD']
flux = tess['FLUX']

And now you can plot the time-series light curve. Once the figure has been created you can interact with it using the toolbar on the left:

<div style="vertical-align:middle;margin:20px">
<span><img src="icon_zoom.png" style="display:inline"> Zoom to rectangle </span>
    <span><img src="icon_home.png" style="display:inline"> Home (return plot to initial state) </span>
    <span><img src="icon_pan.png" style="display:inline"> Pan (click and drag) </span>
    <span><img src="icon_save.png" style="display:inline"> Save the current view to an image </span>
</div>

The $x$ and $y$ coordinates of the cursor are displayed at the bottom of the plot.

In [ ]:
# Plot the TESS time series
fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 4))
plt.plot(bjd, flux, 'o-', ms=2, lw=0.5, color='#CCCCCC',mfc='black', mec='black')
plt.xlabel('BJD $-$ 2450000 (days)')
plt.ylabel('Flux (normalised)')
plt.title('$TESS$ light curve')
plt.show()

You can clearly see the alternating deep and shallow dips in brightness which are typical of an eclipsing binary.

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">Q1. Zoom in and explore the plot. By measuring the time between a few successive primary (or secondary) eclipses, estimate an approximate period ($P$, in days). Save a copy of the plot for your report illustrating your method.</div>
<br>

### Phasing the light curve

As you saw in the pre-lab reading, you can use the period to create a *phased* or *folded* light curve by dividing the observation times by $P$ and keeping only the decimal remainder, $\phi$:

\begin{align}
\phi = (\mathrm{BJD} - t_{0})/P\,\,\,\mathrm{mod}\,\,\,1
\end{align}

where the modulus operator $\mathrm{mod}$ returns the remainder after dividing by $1$. The phase $\phi$ varies between 0 and 1 as the stars move in their orbits and it is convention to choose a $t_{0}$ which puts the primary eclipse at $\phi = 0$. 

### Period finding

Given the short (~2 hr) duration of the eclipses and the 30 min spacing of the *TESS* observations, your initial choice of $P$ probably wasn't the most accurate.  There are several methods in the literature for finding periods of variable stars. Some find general periodic signals while others are customised for particular types of variation. Let's imagine you feed the *TESS* light curve into one of these tools and receive a period of __0.42948402 days__.

What does the phased light curve look like? Run the cell below to find out.


In [ ]:
# Plot the interactive figure
LightCurve(bjd, flux, P=0.42948402)

<br>This doesn't look right either! What has happened? It looks like the period-finder has found a harmonic of the true period. 

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q2. Use your knowledge of what eclipsing binary light curves are <em>meant</em> to look like (plus your estimate of $P$ from the unphased <em>TESS</em> light curve) to figure out what the true period should be. Once you have a nicely phased light curve, adjust the $t_{0}$ value to line up the primary eclipse with $\phi=0$. You may need to zoom in to get the most accurate value. Note the period and $t_{0}$, and save a copy of the figure to include in your report. <em>(Hint: the true $P$ will be some multiple of 0.42948402 days)</em>
</div>

### Light curve discussion

Unlike some eclipsing binaries which show no variation in their light curves outside of the eclipses, THOR 42 has a broad dip in brightness around the primary eclipse. This is probably due to a large, cool group of star spots on the surface of the larger star which rotates into view just before the eclipse. Since THOR 42 is a compact binary we expect the rotation rate of each star to be the same as the orbital period (this is called *tidal synchronisation*) so the spots will never fall out of phase with the eclipse. This is the same as the Moon always keeping the same face towards the Earth during its orbit.

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q3. Use your phased light curve to answer the following questions. You may wish to refer back to the pre-lab reading and video:  
<br>
<ul>
    <li>Assuming the stars have different radii, what do the shapes of the eclipses tell us about the inclination of the system?</li>
    <li>What do the eclipse depths tell us about the temperatures of the stars?</li>
    <li>What does the phase of the secondary eclipse tell us about the eccentricity of the orbits? <br><em>(Hint: think about what would happen if the orbits were elliptical and we were viewing the system along the short axis of the ellipses)</em></li>
</div>
<br>


-------

# 3. Radial velocity curves


Now that we know the period we can analyse the radial velocity data. We again read it from a text file containing the time of observation (BJD - 2450000), the radial velocities for each star (in km s$^{-1}$), their uncertainties and a flag indicating whether the secondary velocity is good (`0`) or bad (`1`):



In [ ]:
!head RV.txt # Display the first 10 rows of the file

The secondary star velocity errors `E_RV2` are much larger because the secondary star is fainter and the velocities are harder to measure. 

First, load the data from the text file and retain the 'good' measurements:

In [ ]:
# Load the RV data 
rv = Table.read('RV.txt', format='ascii')
t1 = rv['BJD']
rv1 = rv['RV1']
e_rv1 = rv['E_RV1']

# Use only those secondary epochs with good velocities (FLAG = 0)
good = rv['FLAG'] == 0 
t2 = rv['BJD'][good]
rv2 = rv['RV2'][good]
e_rv2 = rv['E_RV2'][good]

This leaves 65 primary and 52 secondary velocities.


Next, plot the radial velocities of both stars versus time:


In [ ]:
fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 4))
plt.axhline(0, color='tab:grey', lw=1)
plt.plot(t1, rv1, 'o', ms=4, color='tab:red', label='Primary star')
plt.plot(t2, rv2, 'o', ms=4, color='tab:blue', label='Secondary star')
plt.xlabel('BJD $-$ 2450000 (days)')
plt.ylabel('Radial velocity (km s$^{-1}$)')
plt.title('WiFeS radial velocities')
plt.legend()
plt.show()

As you can see, the first measurement was made at $\mathrm{BJD}\approx2457319$ (2015 October 23) and the velocities had swapped signs at the time the second measurement was made almost a year later. This immediately identified THOR 42 as a spectroscopic binary and it was observed frequently over the following 18 months, often multiple times per night. 

### Fitting radial velocity models

Both stars will obey Kepler's Laws as they orbit around their shared centre-of-mass. You saw in the pre-lab reading that they will trace out observed radial velocity curves according to the equation:

\begin{align}
v_{r}(t) &= v_{\mathrm{sys}} + K[e\cos\omega + \cos(\omega + \phi)]
\end{align}

where $v_{\mathrm{sys}}$ is the *systemic* (or centre-of-mass) velocity, $e$ is the orbital eccentricity ($e=0$ for circular orbits) and $\omega$ rotates the orbits in the plane of the sky. Each star has its own radial velocity *semi-amplitude*, $K_{1}$ and $K_{2}$ <span style="color:red">(Equation 2 in the pre-lab)</span>:

\begin{align}
K_1 &= \frac{2\pi a_1 \sin i}{P \sqrt{1-e^{2}}} \ \ \mathrm{and}\ \ K_2 = \frac{2\pi a_2 \sin i}{P \sqrt{1-e^{2}}}\\
\end{align}

where $P$ is the period, $i$ is the inclination of the orbit ($i=90^{\circ}$ for perfectly edge on) and $a_{1}$,$a_{2}$ are the semi-major axes of the orbits <span style="color:red">(see Figure 1 in the pre-lab)</span>. We of course cannot be sure of the *true* semi-amplitudes because the orbital inclination $\sin i$ is still unknown, but for an eclipsing binary we know it must be close to $i=90^{\circ}$.

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q4a. Use the figure below to fit velocity curves to the observations with your improved $P$ and $t_0$ values. Adjust the sliders to fit radial velocity models $v_{r}(t)$ to the measurements. The values of $K_1$, $K_2$ and $v_{\mathrm{sys}}$ are in km s$^{-1}$ and $\omega$ is measured in degrees. The numbers overlaid on the plot are the "reduced $\chi^2$ (chi-squared) statistic", which is a measure of how well a particular model fits the observations and their uncertainties. A $\chi_{r}^2$ value close to 1.0 indicates a good fit. It should be possible to obtain $\chi_{r}^2$ values of &lt;1.5 for both sets of velocities. Since the radial velocity observations were taken over a longer time period than the light curve, you may wish to slightly alter your $P$ and $t_0$ values to obtain the best fit.
</div>
<br>


In [ ]:
# Plot the interactive figure
RVCurve(t1, rv1, e_rv1, t2, rv2, e_rv2)


<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q4b. When you are happy with your results, record the values of $K_1$, $K_2$, $v_{\mathrm{sys}}$ (all in km s$^{-1})$, $e$ (dimensionless) and $\omega$ (in degrees), and save a copy of the velocity curves for your report. Is the eccentricity consistent with the estimate you made using the light curve in Q3?
</div>
<br>

We now have everything we need to determine the masses and radii of both stars. Check your values of $P$, $K_1$ and $K_2$ with the demonstrator before moving on with the analysis.

<br>


------------

# 4. Estimating the masses and semi-major axis

### Ratio of the masses

<br>From centre-of-mass arguments, you saw in the pre-lab reading that for two stars of masses $M_1$ and $M_2$:

\begin{align}
K_1 M_1 = K_2 M_2
\end{align}

We can therefore immediately calculate the *mass ratio* of the two stars:

\begin{align}
\frac{M_2}{M_1} = \frac{K_1}{K_2}
\end{align}

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q5. Using your fitted values of $K_1$ and $K_2$, calculate the mass ratio of THOR 42, $M_2/M_1$.
</div>
<br>

### Sum of the masses

We can also calculate the sum of the masses by considering Kepler's Third Law <span style="color:red">(Equation 1 in the pre-lab)</span>, namely:

\begin{align}
\frac{a^3}{P^2} &= \frac{G(M_1 + M_2)}{4\pi^2}
\end{align}

where the smaller star orbits around the larger star with semi-major axis $a=a_1 + a_2$ <span style="color:red">(Figure 1 in the pre-lab)</span>.

By rearranging the expressions for $K_1$ and $K_2$ <span style="color:red">(Equation 2)</span>, we see that:

\begin{align}
a_1 = \frac{K_1 P \sqrt{1-e^2}}{2\pi \sin i} \quad\mathrm{and}\quad a_2 = \frac{K_2 P \sqrt{1-e^2}}{2\pi \sin i} 
\end{align}


<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q6. Derive an expression for $a=a_1+a_2$ in terms of $(K_1,K_2,e,i,P)$ and use this to calculate the separation of the stars in solar radii $(1 R_{\odot} = 6.957\times10^{10}\ \mathrm{cm})$ and astronomical units $(1\ \mathrm{au} = 1.4959\times10^{13}\ \mathrm{cm})$, assuming circular orbits and an inclination of $i=85^{\circ}$. Be careful with units in your calculation: you have measured $K_1$ and $K_2$ in km s$^{-1}$ and $P$ in days.
</div>
<br>

Using the expression for $a$ you just calculated, you can substitute into Kepler's Third Law to get the sum of the masses <span style="color:red">(Equation 4 from the pre-lab)</span>:


\begin{align}
(M_1 + M_2)\sin^3 i = \frac{P}{2\pi G}(1-e^2)^{3/2}(K_1 + K_2)^3
\end{align}

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q7. Calculate the sum of the masses in solar units $(1 M_{\odot} = 1.989\times10^{33}\  \mathrm{g})$, assuming an inclination of $i=85^{\circ}$ and circular orbits. Use the sum and ratio of the masses to calculate individual masses $M_1$ and $M_2$.
</div>
<br>

### Sanity check!

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q8. Do your semi-major axis $a$, period $P$ and masses $M_{1}$,$M_{2}$ satisfy Kepler's Third Law, $a^{3}/P^{2} = M_{1} + M_{2}$, when $a$ is measured in astronomical units, $P$ in years and $M_{1}$,$M_{2}$ in solar masses?
</div>
<br>

-------

# 5. Estimating the radii

### Sum of the radii

Consider our toy model of the light curve from the pre-lab reading, where the yellow star is hotter, larger and more massive than the red one:

<img src="eclipses.png">

You saw in the pre-lab <span style="color:red">(Equation 5)</span> that the sum of the *relative* radii $(R_{1}+R_{2})/a$ is related to the duration of the eclipses $\Delta\phi$ and the orbital inclination $i$ through the following equation:

\begin{align}
(R_{1} + R_{2})/a & = \sqrt{\cos^2 i + \sin^2(\pi \Delta\phi)}
\end{align}

where $a$ is the semi-major axis of the binary from the discussion of Kepler's Third Law.

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q9a. Measure THOR 42's primary and secondary eclipse durations from its phased <em>TESS</em> light curve and calculate an average duration $\Delta\phi$. Be consistent in how you define the beginning and end of each eclipse, and include an annotated light curve in your report illustrating your method. Use your $\Delta\phi$ measurement to calculate the sum of the relative radii $(R_{1}+R_{2})/a$ assuming $i=85^{\circ}$. Remember to convert $i$ to radians (or $\pi \Delta\phi$ to degrees) in order to correctly apply the equation. 
</div>
<br>

<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q9b. Convert this to the sum of the <em>absolute</em> radii $(R_1 + R_2)$ using your value for the semi-major axis $a$ calculated above.
</div>
<br>



### Ratio of the radii

Now we can find the *ratio* of the radii from the eclipse depths. Recall from the pre-lab reading that the surface flux ratio $F_{1}/F_{2}$ is related to the eclipse depths by: 

\begin{align}
\frac{B_0-B_1}{B_0-B_2} & = \frac{F_1}{F_2} \\
\end{align}

and you know from the definition of luminosity ($L=4\pi R^2 F = 4\pi R^2 \sigma T_{\mathrm{eff}}^4$) that the radii are related to the surface fluxes and luminosity as follows:

\begin{align}
\frac{R_2}{R_1} = \sqrt{\frac{L_2}{L_1}\frac{F_1}{F_2}}
\end{align}

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q10. Carefully measure both eclipse depths from the phased light curve and calculate the surface flux ratio $F_{1}/F_{2}$. As the <em>TESS</em> fluxes have already been normalised, you can assume $B_0 = 1.0$ for both eclipses (i.e. ignore the broad dip around the primay eclipse due to the spots). 
</div>
<br>

### Luminosity ratio

Now all we need is the ratio of the luminosities. Unlike main sequence stars which have luminosities that scale with mass as $L\propto M^{3.5}$, young <em>pre-</em>main sequence stars are still contracting and evolve to lower luminosities more quickly at higher masses. This means at a given age their luminosity-mass relation is *shallower* than for main sequence stars. At an age of $\sim$20 Myr, theoretical models show that lumonisity scales as approximately $M^{1.4}$ for masses <$1 M_{\odot}$.

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q11. Use the fact that $L\propto M^{1.4}$ for these stars to find the expected luminosity ratio $L_2/L_1$. How does it compare to the main sequence value?
</div>
<br>

<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q12. Use your values of $L_2/L_1$ and $F_1/F_2$ to calculate the ratio of the radii, $R_2/R_1$. By combining this with the sum of the radii calculate the individual values $R_1$ and $R_2$.
</div>
<br>

-----
# 6. Age of THOR 42

Young, low-mass stars are not yet in hydrostatic equilibrium, and are slowly contracting down to their  main sequence radii. The regularity of this contraction means we can use the measured radius to estimate an age for the star by comparing its progress to theoretical stellar [*isochrones*](https://ui.adsabs.harvard.edu/abs/2015A%26A...577A..42B/abstract), which join together stellar models of constant age.  Both members of THOR 42 should have similar ages (co-evality) as they were born together in the same star-forming event.

<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q13. Add your mass and radius measurements to the figure below. Adjust the isochrone to estimate individual ages for both stars. Are they approximately co-eval? Do your ages agree with the $24\pm4$ Myr age of the 32 Orionis Moving Group? Remember to include a copy of this figure in your report.
</div>
<br>

In [ ]:
# Plot the interactive figure
MassRadiusDiagram()

-----
# 7. Conclusions



<br>
<div style="font-weight:bold; background-color:#ffcccb; padding:10px; border:1px solid red; border-radius:5px">
    Q14. In a short paragraph, briefly summarise your analysis and findings for THOR 42, stating your main numerical results and including answers to the following questions:<br>
<ul>
    <li>Which physical quantities can be derived from the <em>light curve</em> of an eclipsing binary? What about the <em>radial velocity curves</em>? </li>
<li>What were the major sources of uncertainty in your analysis? This can include limitations of the data or techniques, as well as any assumptions made.</li>
<li>How might you improve your parameter estimates using different techniques and/or observations?</li>
</ul>
</div>
<br>

*Extra for those who are interested:* You may wish to compare your findings to recent published values for THOR 42 made using much the same observational data. Table 6 in [this paper in the Monthly Notices of the Royal Astronomical Society](https://arxiv.org/pdf/1911.05925.pdf) contains estimates of the system parameters and their uncertainties. Some of the authors may be familiar :) How do your values and analysis methods compare?

<br><br><br>
